In [ ]:
%%writefile add.cu
#include<bits/stdc++.h>
#include<cstdlib>
using namespace std;

#define N 1000000

__global__ void add(int *a,int *b,int *c){
  int idx = blockDim.x*blockIdx.x + threadIdx.x;
  if(idx<N)
    c[idx] = a[idx] + b[idx];
}

int main(){
  int *a,*b,*c;
  int *da,*db,*dc;
  int size = N*sizeof(int);

  //host 
  a = (int *)malloc(size);
  b = (int *)malloc(size);
  c = (int *)malloc(size);

  //device 
  cudaMalloc(&da,size);
  cudaMalloc(&db,size);
  cudaMalloc(&dc,size);

  for(int i=0;i<N;i++){
    a[i] = i;
    b[i] = i*2;
  }


  cudaMemcpy(da,a,size,cudaMemcpyHostToDevice);
  cudaMemcpy(db,b,size,cudaMemcpyHostToDevice);

  add<<<(N+255)/256,256>>>(da,db,dc);

  cudaMemcpy(c,dc,size,cudaMemcpyDeviceToHost);

  cout <<"c[0] ="<<c[0]<<" , c["<<N-1<<"] = "<<c[N-1];

  free(a);
  free(b);
  free(c);

  cudaFree(da);
  cudaFree(db);
  cudaFree(dc);
  return 0;
}

Overwriting add.cu


In [ ]:
!nvcc add.cu -o add

In [ ]:
!./add


c[0] =0 , c[999999] = 2999997

In [ ]:
!nvprof ./add

==896== NVPROF is profiling process 896, command: ./add
==896== Profiling application: ./add
c[0] =0 , c[999999] = 2999997==896== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   52.76%  1.7158ms         1  1.7158ms  1.7158ms  1.7158ms  [CUDA memcpy DtoH]
                   45.78%  1.4888ms         2  744.42us  712.34us  776.50us  [CUDA memcpy HtoD]
                    1.45%  47.199us         1  47.199us  47.199us  47.199us  add(int*, int*, int*)
      API calls:   97.20%  255.40ms         3  85.133ms  100.97us  255.18ms  cudaMalloc
                    2.05%  5.3732ms         3  1.7911ms  933.64us  3.4792ms  cudaMemcpy
                    0.40%  1.0485ms         1  1.0485ms  1.0485ms  1.0485ms  cuDeviceGetPCIBusId
                    0.25%  659.53us         3  219.84us  158.29us  251.52us  cudaFree
                    0.07%  189.31us       101  1.8740us     200ns  76.431us  cuDeviceGetAttribute
                    0.